In [7]:
import cv2
import numpy as np
from math import pi, cos, sin, exp
import keyboard
from time import time
from values import *

In [24]:
def initXandXd():
    x = np.empty((0, 2), int) # actual position
    xd = np.empty((0, 2), int) # desired position
    return x, xd

def resetXAndXd():
    x, xd = initXandXd()

    x = np.append(x, np.array([[int(WIDTH/2), int(HEIGHT/2)]]), axis=0) # actual position
    xd = np.append(xd, np.array([[int(WIDTH/2 + WIDTH/4), int(HEIGHT/2 + HEIGHT/4)]]), axis=0) # desired position
    return x, xd

x, xd = initXandXd()
x, xd = resetXAndXd()
Fh = np.array([]) # force applied to robot

In [27]:
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# Shared Variables
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%
i = len(xd) - 1 # number of sampling times
e = xd - x # array of errors between all positions of x and xd

# for my code the error is considered as the affecting force, just bcz it's the simplest way to go about it
Fh = e.copy()

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# Trajactory Adaptation
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# Operation - Trajactory Adaptation
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Epsilon1 = K1 * e*e # performance evaluation index

# Index function J
J = 0
for j in range(0, i):
    J = J + Epsilon1[j] * np.exp( - Mu1 * (i - j) * dT )

Alpha1 = 1 - np.exp( -v1 * J ) # variable to be designed according to a specific system to adjust the user's interaction profile
Beta1 = M1 * Alpha1 # an open parameter to regulate the interaction force

# final relation between x, xd and Fh
xd_trajectory_adaptation = x + Beta1 * Fh # oh we experienced disturbing force Fh, let's see where we should move next to reduce Fh, we find xd


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# Adaptive impedance Control
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# we must differentiate X axis and Y axis separately from each other then put them back together
A1 = np.diff([ A1[0] for A1 in e ])
A2 = np.diff([ A2[1] for A2 in e ])
e_derivative = np.empty((0, 2), int)
e_derivative = np.insert(e_derivative, 0, e[0], axis=0)
for i in range(0, len(A1)):
    e_derivative = np.insert(e_derivative, 0, np.array([A1[i], A2[i]]), axis=0)
# because the python differentiation doesn't insert the first element that start from zero we add it

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# Operation - Adaptive impedance Control
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Epsilon2 = K2 * e*e # performance evaluation index

# Index function J2
J2 = 0
for j in range(0, i):
    J2 = J2 + Epsilon2[j] * np.exp( - Mu2 * (i - j) * dT )

Alpha2 = np.exp( -v2 * J2 ) # variable to be designed according to a specific system to adjust the user's interaction profile

Beta2 = M2 * Alpha2 # paramter to regulate the stiffness of the impedence controller
deltaK = Beta2 * abs(Fh)
Kc = Kc0 + deltaK # variable stiffness
Fim = Kc * e + Kd * e_derivative # it's u, input of the impedance control

# final relation between x, xd and Fh
xd_impedence_control = x + Beta2 * Fh # oh we experienced disturbing force Fh, let's see where we should move next to reduce Fh, we find xd



# TODO: you're only using xd_impedence_control in the line before   
x = np.append(x, np.array([[ int(xd_impedence_control[0][0]), int(xd_impedence_control[0][1]) ]]), axis=0) # just keep all coordinates as int
xd = np.append(xd, np.array([xd[len(xd)-1]]) , axis=0) # i guess we keep desiring the same spot until we reach it?


In [31]:
Fh+100

array([[225, 250],
       [100, 100],
       [100, 100]])